## Cisors 2000

In [26]:
import os
import cv2
import cv2

#### Configuration

In [49]:
INPUT_IMAGE_DIR = 'Test/images'
INPUT_LABEL_DIR = 'Test/labels'
OUTPUT_IMAGE_DIR = 'Output/images'
OUTPUT_LABEL_DIR = 'Output/labels'

PRINT_BBOX_DIR = 'BBOX30'

ROWS = 2  # Découpage vertical
COLS = 2  # Découpage horizontal

# Préparation des dossiers de sortie
os.makedirs(OUTPUT_IMAGE_DIR, exist_ok=True)
os.makedirs(OUTPUT_LABEL_DIR, exist_ok=True)
os.makedirs(PRINT_BBOX_DIR, exist_ok=True)

Format  des bounding boxes :  ``class`` ``x_center`` ``y_center`` `width` `height` (normalisés entre 0 et 1).

In [50]:
def convert_bbox(bbox, orig_w, orig_h, x_offset, y_offset, crop_w, crop_h):
    """
    Convertit les coordonnées bbox d'origine en bbox relative au crop.

    Args:
        bbox (tuple[float, float, float, float]): données initiales de la bbox (x_center, y_center, width, height)
        orig_w (int): largeur de l'image d'origine en px
        orig_h (int): hauteur de l'image d'origine en px
        x_offset (int): décalage sur x du crop par rapport à l'image d'origine en px
        y_offset (int): décalage sur y du crop par rapport à l'image d'origine en px
        crop_w (int): largeur du crop en px
        crop_h (int): hauteur du crop en px

    Returns:
        tuple[float, float, float, float]: données convertit de la bbox relativement au crop
    """

    cx, cy, w, h = bbox

    # Conversion des coordonnées normalisées en cordonnées absolues (px)
    abs_cx = cx * orig_w
    abs_cy = cy * orig_h
    abs_w = w * orig_w
    abs_h = h * orig_h

    bbox_area = abs_w * abs_h     # Aire de la bbox
    ratio_min = 0.3       # Ratio min pour une bbox valide


    x1 = abs_cx - abs_w / 2  # Coin Gauche
    y1 = abs_cy - abs_h / 2  # Coin Haut
    x2 = abs_cx + abs_w / 2  # Coin Droit
    y2 = abs_cy + abs_h / 2  # Coin Bas


    # Détermination des coupes
    crop_x1, crop_y1 = x_offset, y_offset
    crop_x2, crop_y2 = x_offset + crop_w, y_offset + crop_h

    # On détecte si une bbox est partiellement ou entièrement dans le crop
    inter_x1 = max(x1, crop_x1)
    inter_y1 = max(y1, crop_y1)
    inter_x2 = min(x2, crop_x2)
    inter_y2 = min(y2, crop_y2)

    if inter_x1 >= inter_x2 or inter_y1 >= inter_y2:
        return None  # Le rectangle d'intersection est vide (la bbox n'est pas dans le crop)

    # Nouvelle box dans le crop
    new_cx = (inter_x1 + inter_x2) / 2 - crop_x1   # on fait moins crop_x1 pour repositionner dans le repère local du crop
    new_cy = (inter_y1 + inter_y2) / 2 - crop_y1
    new_w = inter_x2 - inter_x1
    new_h = inter_y2 - inter_y1

    #print(new_w, new_h, bbox_area)

    if ( new_h * new_w ) / bbox_area < ratio_min:
        #print('je suis rentré dans la condition')
        return None  # La bbox est trop coupée

    # Normalisation pour le format YOLO
    return [
        new_cx / crop_w,
        new_cy / crop_h,
        new_w / crop_w,
        new_h / crop_h
    ]

In [51]:
for img_file in os.listdir(INPUT_IMAGE_DIR):
    if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    img_path = os.path.join(INPUT_IMAGE_DIR, img_file)
    label_path = os.path.join(INPUT_LABEL_DIR, os.path.splitext(img_file)[0] + '.txt')

    img = cv2.imread(img_path)
    orig_h, orig_w = img.shape[:2] # On récupère la taille de l'image en px
    crop_h = orig_h // ROWS
    crop_w = orig_w // COLS

    # Chargement des labels YOLO
    bboxes = []
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                cls, x, y, w, h = map(float, parts)
                bboxes.append((int(cls), [x, y, w, h]))

    # Découpe de l’image
    for i in range(ROWS):
        for j in range(COLS):
            x_offset = j * crop_w
            y_offset = i * crop_h

            crop = img[y_offset:y_offset + crop_h, x_offset:x_offset + crop_w]
            crop_name = f"{os.path.splitext(img_file)[0]}_{i}_{j}.jpg"
            crop_label = f"{os.path.splitext(img_file)[0]}_{i}_{j}.txt"

            cv2.imwrite(os.path.join(OUTPUT_IMAGE_DIR, crop_name), crop)

            # On construit le fichier label
            new_labels = []
            for cls, bbox in bboxes:
                new_bbox = convert_bbox(bbox, orig_w, orig_h, x_offset, y_offset, crop_w, crop_h)
                if new_bbox:
                    new_labels.append(f"{cls} {' '.join(f'{v:.6f}' for v in new_bbox)}")

            # On écrit le fichier label
            with open(os.path.join(OUTPUT_LABEL_DIR, crop_label), 'w') as f:
                f.write('\n'.join(new_labels) + '\n' if new_labels else '')

224.67000000000002 101.15999999999998 22727.617199999997
72.3 107.59999999999997 7779.48
28.350000000000023 203.11999999999998 31282.263199999998
je suis rentré dans la condition
209.96000000000004 58.565 12297.3572
107.44999999999999 85.65 22439.857999999997
209.56000000000003 39.964999999999975 35342.294
je suis rentré dans la condition
95.07 74.04 7038.9828
172.96000000000004 162.03000000000003 28024.7088
16.329999999999927 160.70999999999998 2693.7968
136.93000000000006 163.165 26743.798300000002
70.09000000000003 203.11999999999998 31282.263199999998
209.62000000000012 66.28500000000003 36237.0094
98.34999999999991 66.66499999999999 6556.9945
28.350000000000023 114.65999999999997 31282.263199999998
je suis rentré dans la condition
107.44999999999999 123.18999999999994 22439.857999999997
209.56000000000003 128.685 35342.294
20.97999999999996 95.53999999999996 8893.2648
je suis rentré dans la condition
16.329999999999927 4.25 2693.7968
je suis rentré dans la condition
136.9300000000

#### Pour visualiser les bbox

In [30]:
def draw_yolo_bboxes(image_path, label_path, class_names=None):
    # Charger l'image
    image = cv2.imread(image_path)
    h, w = image.shape[:2]

    # Lire les annotations YOLO
    with open(label_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        parts = line.strip().split()
        class_id = int(parts[0])
        x_center, y_center, bbox_w, bbox_h = map(float, parts[1:])

        # Convertir en coordonnées absolues
        x1 = int((x_center - bbox_w / 2) * w)
        y1 = int((y_center - bbox_h / 2) * h)
        x2 = int((x_center + bbox_w / 2) * w)
        y2 = int((y_center + bbox_h / 2) * h)

        # Dessiner la bbox
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Afficher le nom de la classe si fourni
        if class_names:
            label = class_names[class_id]
            cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (0, 255, 0), 1, cv2.LINE_AA)

    output_path = os.path.join(PRINT_BBOX_DIR, f"bbox_{os.path.basename(image_path)}")
    cv2.imwrite(output_path, image)

##### Visualiser toutes les images originales

In [46]:
for image in os.listdir(INPUT_IMAGE_DIR):
    draw_yolo_bboxes(f'{INPUT_IMAGE_DIR}/{image}', f'{INPUT_LABEL_DIR}/{os.path.splitext(image)[0]}.txt')

![image_originale](BBOX50/bbox__99191421_gettyimages-801124046_jpg.rf.cf018c6930c40b99edb7e54d07d85c6c.jpg)

##### Visualiser toutes les images coupées

In [52]:
for image in os.listdir(OUTPUT_IMAGE_DIR):
    draw_yolo_bboxes(f'{OUTPUT_IMAGE_DIR}/{image}', f'{OUTPUT_LABEL_DIR}/{os.path.splitext(image)[0]}.txt')


Résultat des découpes avec un ration de 50

![decoupe1](BBOX50/bbox__99191421_gettyimages-801124046_jpg.rf.cf018c6930c40b99edb7e54d07d85c6c_0_0.jpg)
![decoupe2](BBOX50/bbox__99191421_gettyimages-801124046_jpg.rf.cf018c6930c40b99edb7e54d07d85c6c_0_1.jpg)
![decoupe3](BBOX50/bbox__99191421_gettyimages-801124046_jpg.rf.cf018c6930c40b99edb7e54d07d85c6c_1_0.jpg)
![decoupe4](BBOX50/bbox__99191421_gettyimages-801124046_jpg.rf.cf018c6930c40b99edb7e54d07d85c6c_1_1.jpg)

Résultat des découpes avec un ration de 30

![decoupe1](BBOX30/bbox__99191421_gettyimages-801124046_jpg.rf.cf018c6930c40b99edb7e54d07d85c6c_0_0.jpg)
![decoupe2](BBOX30/bbox__99191421_gettyimages-801124046_jpg.rf.cf018c6930c40b99edb7e54d07d85c6c_0_1.jpg)
![decoupe3](BBOX30/bbox__99191421_gettyimages-801124046_jpg.rf.cf018c6930c40b99edb7e54d07d85c6c_1_0.jpg)
![decoupe4](BBOX30/bbox__99191421_gettyimages-801124046_jpg.rf.cf018c6930c40b99edb7e54d07d85c6c_1_1.jpg)